# Tailgating
A numerical example, demonstrating the tailgating procedure in the case of computing derivatives of a molecular energy function with respect to some set of nuclear coordinates.

In [1]:
import tailgating as tg
import pennylane as qml
import pennylane.numpy as np
from pennylane import qchem
import scipy
from tqdm.notebook import tqdm

### Step 1. Performing ADAPT-VQE

In this Notebook, we will attempt to compute a second derivative of the energy function corresponding to a BeH$_2$ molecule. We begin by declaring some basic information about the molecule:

In [2]:
# Basic information
symbols = ["Be", "H", "H"]
geometry = tg.angs_bohr * np.array([[0.0000000000, 0.0000000000, 0.0000000000], [0.0000000000, -1.3164290143, 0.0000000000], [0.0000000000, 1.3164290143, 0.0000000000]])

# Specifies the electrons and orbitals
electrons, active_electrons = 6, 4
orbitals, active_orbitals = 7, 6
wires = list(range(2 * active_orbitals))

# Defines the molecule and the active space
mol = qml.hf.Molecule(symbols, geometry)
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=6)

We will compare calculations done with ADAPT-VQE, with and without the tailgating procedure, and demonstrate that the non-tailgated circuit yields inaccurate derivatives while those yielded from tailgating are highly accurate. The first step is to perform ADAPT-VQE, using a gate pool of single and double excitations:

In [3]:
# Generates the gate pool to use for ADAPT-VQE
gate_pool = tg.gate_pool(active_electrons, active_orbitals)

# Generates the sparse molecular Hamiltonian (used in the VQE simulations)
H = tg.hamiltonian_sparse(mol, wires, core, active)(geometry.flatten())

# Other information: the device, Hartree-Fock state, optimizer, and numbers of optimization steps
dev = qml.device('default.qubit', wires=wires)
hf_state = qchem.hf_state(active_electrons, 2 * active_orbitals)
optimizer = qml.GradientDescentOptimizer(stepsize=0.1)
max_steps, vqe_steps = 20, 100

# Performs the original ADAPT-VQE procedure
original_seq, original_params = tg.adapt_vqe(H, dev, gate_pool, hf_state, optimizer, max_steps, vqe_steps, bar=True)

/Users/jackceroni/Library/Python/3.9/lib/python/site-packages/PennyLane-0.21.0.dev0-py3.9.egg/pennylane/_grad.py:95: UserWarning: Starting with PennyLane v0.21.0, when using Autograd, inputs have to explicitly specify requires_grad=True (or the argnum argument must be passed) in order for trainable parameters to be identified.
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Since we are working with a simulator, we have direct access to the state vector, and can determine how close it is to the true ground state:

In [5]:
# Circuit yielded from ADAPT-VQE
def circuit(params):
    qml.BasisState(hf_state, wires=wires)
    for p, gate in zip(params, original_seq):
        gate(p)

# State vector yielded from ADAPT-VQE circuit
state = tg.compute_state(circuit, dev, original_params)

# Returns the dot product between the true ground state of H and the prepared ground state
print(abs(np.dot(scipy.sparse.linalg.eigs(H.matrix)[1].T[0], state)))

0.9998451336687443


As can be seen, the magnitude of the dot product is very close to one, implying that the prepared state is very close to the true ground state. However, as we will soon demonstrate, this does not necessarily mean that our circuit will yield accurate energy derivatives.

### Step 2. Computing the energy derivative with the non-tailgated circuit

The next step is to calculate the second-order energy derivative using the non-tailgated circuit. Recall that the formula for such a derivative is given by:

$$
\frac{\partial^2 E_k(\textbf{R})}{\partial R_i \partial R_j} = \displaystyle\sum_{a} \Big[ \frac{\partial \boldsymbol\theta^{k}_a(\textbf{R})}{\partial R_i} \frac{\partial}{\partial \theta_a} \frac{\partial E_k(\textbf{R})}{\partial R_j} \Big] + \Big\langle \psi_{k}(\boldsymbol\theta^{k}(\textbf{R})) \Big| \frac{\partial^2 H(\textbf{R})}{\partial R_i \partial R_j} \Big| \psi_{k}(\boldsymbol\theta^{k}(\textbf{R})) \Big\rangle
$$

where $|\psi_k(\theta)\rangle = U_k(\theta) |0\rangle$ is a variational ansatz which is used to prepare the $k$-th excited state, and $\boldsymbol{\theta}^k(\textbf{R})$ are the variational parameters of $U_k(\theta)$ which prepare the $k$-th energy eigenstate at coordinates $\textbf{R}$. In addition, by Feynman-Hellmann theorem, we have

$$
\frac{\partial E_k(\textbf{R})}{\partial R_j} = \Big\langle \psi_k( \boldsymbol\theta^{k}(\textbf{R})) \Big| \frac{\partial H(\textbf{R})}{\partial R_j} \Big| \psi_k(\boldsymbol\theta^{k}(\textbf{R})) \Big\rangle
$$

Finally, we can determine the derivatives of the optimal parameters $\boldsymbol{\theta}^{k}$ with the response equation:

$$
\displaystyle\sum_{a} \frac{\partial}{\partial \theta_b} \frac{\partial E_k(\textbf{R})}{\partial \theta_a} \frac{\partial \boldsymbol{\theta}^{k}_a(\textbf{R})}{\partial R_i} = -\frac{\partial}{\partial \theta_b} \frac{\partial E_k(\textbf{R})}{\partial R_i}
$$

We begin by computing the first derivatives of the molecular Hamiltonian:

In [6]:
# Generates first and second derivatives of the Hamiltonian
H1 = tg.generate_d_hamiltonian(mol, wires, core, active)(geometry.flatten())

  0%|          | 0/9 [00:00<?, ?it/s]

/Users/jackceroni/Library/Python/3.9/lib/python/site-packages/autohf-0.0.1-py3.9.egg/autohf/hartreefock.py:209: RuntimeWarning: divide by zero encountered in true_divide


Next, we compute the matrix of second derivatives (with respect to variational parameters) of the energy function:

In [53]:
# Non-sparse Hamiltonian for computing second derivatives
h_new = tg.hamiltonian(mol, wires, core, active)(geometry.flatten())

# Energy function
@qml.qnode(dev)
def energy_fn(params):
    circuit(params)
    return qml.expval(h_new)

# Parameter Hessian
hessian = compute_parameter_hessian(energy_fn, np.array(original_params), bar=True)

  0%|          | 0/20 [00:00<?, ?it/s]

Finally, we can compute the second derivative of the Hamiltonian, and the desired energy derivative. In particular, we will compute $\frac{\partial E}{\partial R_1^2}$, where $R_1$ is the $y$-coordinate of the Be atom:

In [54]:
# Hamiltonian second derivative
vec = np.array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
H2 = tg.dd_hamiltonian(mol, wires, core, active)(geometry.flatten(), vec, vec)

# Energy derivative
energy_derivative = tg.compute_second_derivative([1, 1], H1, H2, hessian, circuit, dev, original_params, diff_method="best")

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

The value of the energy derivative is then:

In [55]:
print("Energy derivative = {}".format(energy_derivative))

Energy derivative = 1.6190857047649865


This is quite different from the value 0.411577257 predicted by GAMESS (software for performing quantum chemistry calculations with classical methods). 

### Step 3. Performing tailgated ADAPT-VQE

We will now demonstrate that tailgating yields the correct derivative. Taking into account symmetry of the molecule, we will only tailgate with respect to derivatives of the Be atom along the $x$-axis, and the top hydrogen atom along the $x$ and $y$-axes.

In [56]:
# Performs the tailgating procedure
hamiltonians = [H1[0], H1[3], H1[4]]

# A method for tailgating a circuit
def tailgate(dH, dev, circuit, operator_pool, params, tol=1e-6):
    """Performs the tail-gating procedure"""
    gates = []
    bar = tqdm(operator_pool)

    for op in bar:
        @qml.qnode(dev)
        def cost_fn(param):
            circuit(params)
            op(param)
            return qml.expval(dH)
        if abs(qml.grad(cost_fn)(0.0)) > tol:
            gates.append(op)
    return gates

# Gates added to the circuit through tailgating
added_gates = []
for dh in hamiltonians:
    gates = tailgate(dh, dev, circuit, gate_pool, original_params)
    added_gates.extend(gates)

added_gates = set(added_gates)

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

Now, we can construct a new, tailgated circuit:

In [57]:
# Creates the new gate pool
new_seq = original_seq + list(added_gates)
new_params = original_params + [0.0 for _ in range(len(added_gates))]

# Circuit yielded from tailgating
def new_circuit(params):
    qml.BasisState(hf_state, wires=wires)
    for p, gate in zip(params, new_seq):
        gate(p)

Once again, we compute the desired energy derivative:

In [58]:
# Energy function
@qml.qnode(dev)
def new_energy_fn(params):
    new_circuit(params)
    return qml.expval(h_new)

# Parameter Hessian
new_hessian = compute_parameter_hessian(new_energy_fn, np.array(new_params), bar=True)

  0%|          | 0/66 [00:00<?, ?it/s]

In [59]:
# Energy derivative
new_energy_derivative = tg.compute_second_derivative([1, 1], H1, H2, new_hessian, new_circuit, dev, new_params, diff_method="best")

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

which gives us the following value:

In [60]:
print("Energy derivative = {}".format(new_energy_derivative))

Energy derivative = 0.4113317834975785


Much better! Our answer agrees with GAMESS to three decimal places!